# Fetch Data



### Set env variables

assumes that `SOCRATA_APP_TOKEN`, `SOCRATA_API_KEY_ID`, and `SOCRATA_API_KEY_SECRET` are set in `.env`

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

### Load Packages

In [ ]:
import os
import sys
import pdb
import censusdata
import pandas as pd

INGESTION_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, INGESTION_PATH)

from ingestion import fetch
from ingestion import config

ImportError: cannot import name 'OUTPUT_DIR' from 'ingestion.config' (c:\Users\gorav\GitHub\nyc-311-service-requests\src\ingestion\config.py)

### Full Pull of Data
Takes about 48 minutes

In [ ]:
# await fetch.fetch_all_data(save = True)

### Incremental Pull of Data

In [ ]:
# await fetch.fetch_current_month(save = True)

### ACS Census Data

In [ ]:


# =========================
# 1. Load the block group shapefile
# =========================
bg_shapefile = os.path.abspath(os.path.join(INGESTION_PATH, 'resources', "tl_2022_36_bg"))  # replace with your path
gdf_bg = gpd.read_file(bg_shapefile)

# Keep only NYC counties
nyc_counties = ["005", "047", "061", "081", "085"]  # Bronx, Kings, Manhattan, Queens, Richmond
gdf_bg = gdf_bg[gdf_bg['COUNTYFP'].isin(nyc_counties)].copy()

# Block group data is only available from 2012+ in ACS 5-year
# For 2010-2011, you'd need to use tract-level data or download from data.census.gov
years = list(range(2013, 2024))  # 2012–2023
acs_var = ['B01003_001E']  # total population

all_data = []

# Download data for each NYC county separately to avoid API issues
nyc_counties_list = ["005", "047", "061", "081", "085"]

for year in years:
    print(f"Downloading ACS 5-year data for {year}...")
    year_data = []
    
    for county in nyc_counties_list:
        try:
            print(f"  - County {county}...")
            data = censusdata.download(
                src='acs5',
                year=year,
                geo=censusdata.censusgeo([('state', '36'), ('county', county), ('block group', '*')]),
                var=acs_var
            )
            year_data.append(data)
        except Exception as e:
            print(f"    Error downloading county {county} for year {year}: {e}")
            continue
    
    if year_data:
        combined = pd.concat(year_data)
        combined = combined.reset_index()
        combined['year'] = year
        all_data.append(combined)

# Combine all years
df_pop = pd.concat(all_data, ignore_index=True)

# Parse GEOID from censusgeo index
def geo_to_geoid(geo):
    parts = geo.params()  # params is a method, not a property()
    return parts[0][1] + parts[1][1] + parts[2][1] + parts[3][1]

df_pop['GEOID'] = df_pop['index'].apply(geo_to_geoid)
df_pop.rename(columns={'B01003_001E': 'population'}, inplace=True)
df_pop = df_pop[['GEOID', 'population', 'year']]

# =========================

  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...


In [ ]:
 os.path.abspath(os.path.join(config.LOCAL_OUTPUT_DIR, "acs_population","landing"))

AttributeError: module 'ingestion.config' has no attribute 'LOCAL_OUTPUT_DIR'

In [ ]:
LOCAL_OUTPUT_DIR

In [ ]:
df_pop.to_csv()

,GEOID,population,year
0,360050023002,733,2013
1,360050039004,1042,2013
2,360050054002,3138,2013
3,360050062003,1469,2013
4,360050068001,1726,2013
...,...,...,...
72674,360850319022,1779,2023
72675,360850319023,1659,2023
72676,360850323000,0,2023
72677,360850323001,1078,2023
